In [ ]:
%load_ext dotenv
%dotenv

import os
from openai import AzureOpenAI
from azure.identity import ClientSecretCredential, get_bearer_token_provider
# Azure OpenAI Service Integration with LangChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage


In [ ]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2025-01-01-preview",
    temperature=0,
    max_tokens=200,
    timeout=None,
    max_retries=6
)

In [ ]:
chat_template_tools = ChatPromptTemplate.from_template('''
                                                      What is the most important 5 tools for {job}?
                                                      Answer only by listing available tools.
                                                      ''')
chat_template_strategy = ChatPromptTemplate.from_template('''
                                                      What is the best strategy for learning and mastering {tools}?
                                                      ''')

In [ ]:
string_parser = StrOutputParser()

In [ ]:
chain_tools = chat_template_tools | llm | string_parser
chain_strategy = chat_template_strategy | llm | string_parser

In [ ]:
print(chain_tools.invoke({'job': 'AI Engineer'}))

In [ ]:
res = chain_tools.invoke({'job': 'AI Engineer'})
chain_strategy.invoke({'tools': res})

In [ ]:
chain_tools = chat_template_tools | llm | string_parser | {'tools': RunnablePassthrough()}
chain_strategy = chat_template_strategy | llm | string_parser

In [ ]:
chain_tools.invoke({'job': 'AI Engineer'})

In [ ]:
chain_combined = chain_tools | chain_strategy

In [ ]:
chain_combined.invoke({'job': 'AI Engineer'})

LangChain の状態を可視化

In [ ]:
chain_combined.get_graph().print_ascii()

## Runnable Parallel
テンプレートで使用する変数が共通の場合に、テンプレートを渡して並列実行できる

Sequencial に実行するより実行時間が短くて済む

In [ ]:
from langchain_core.runnables import RunnableParallel
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

In [ ]:
chain_books = chat_template_books | llm | string_parser
chain_projects = chat_template_projects | llm | string_parser

In [40]:
chain_parallel = RunnableParallel({'books': chain_books, 'projects': chain_projects})
chain_parallel.invoke({'programming language': 'Python'})

{'books': '1. *Fluent Python* by Luciano Ramalho  \n2. *Effective Python: 90 Specific Ways to Write Better Python* by Brett Slatkin  \n3. *Python Cookbook* by David Beazley and Brian K. Jones  ',
 'projects': '1. Personal Finance Tracker  \n2. Weather Dashboard with API Integration  \n3. Multiplayer Chat Application using Sockets  '}

In [ ]:
chain_parallel.get_graph().print_ascii()

In [39]:
chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [42]:
chain_time1 = (RunnableParallel({'books':chain_books, 
                                'projects':chain_projects}) 
              | chat_template_time 
              | llm 
              | string_parser
             )

In [43]:
chain_time1.invoke({'programming language': 'Python'})

'The time required to complete the literature and projects depends on several factors, including your current skill level, the amount of time you can dedicate daily, and your familiarity with the tools and libraries involved. Below is a rough estimate for an intermediate-level programmer:\n\n---\n\n### **Literature Completion Time**\n1. **Fluent Python by Luciano Ramalho**  \n   - **Length**: ~800 pages  \n   - **Focus**: Advanced Python concepts, idiomatic Python, and best practices.  \n   - **Estimated Time**: 2–3 months (if you read ~10 pages per day and spend time experimenting with the examples).  \n\n2. **Effective Python: 90 Specific Ways to Write Better Python by Brett Slatkin**  \n   - **Length**: ~300 pages  \n   - **Focus**: Practical tips and techniques for writing better Python code.  \n   - **Estimated Time**: 1–1.5 months (if you read ~10 pages per day'

In [44]:
chain_time1.get_graph().print_ascii()

               +-------------------------------+                 
               | Parallel<books,projects>Input |                 
               +-------------------------------+                 
                    ***                  ***                     
                ****                        ****                 
              **                                **               
+--------------------+                   +--------------------+  
| ChatPromptTemplate |                   | ChatPromptTemplate |  
+--------------------+                   +--------------------+  
           *                                        *            
           *                                        *            
           *                                        *            
  +-----------------+                     +-----------------+    
  | AzureChatOpenAI |                     | AzureChatOpenAI |    
  +-----------------+                     +-----------------+    
          

In [45]:
chain_time2 = ({'books':chain_books, 
                                'projects':chain_projects} 
              | chat_template_time 
              | llm 
              | string_parser
             )

In [46]:
chain_time2.invoke({'programming language': 'Python'})

'The time required to complete the literature and projects depends on several factors, including your current skill level, the amount of time you can dedicate daily, and your familiarity with the topics covered in the books and projects. Below is a rough estimate for an intermediate-level programmer:\n\n---\n\n### **Literature Completion Time**\n1. **Fluent Python by Luciano Ramalho**  \n   - **Length**: ~800 pages  \n   - **Difficulty**: Intermediate to advanced concepts, focusing on Pythonic idioms and best practices.  \n   - **Estimated Time**: 2–3 months (if you dedicate 1–2 hours per day).  \n     - This book is dense and requires time to absorb the concepts and practice the examples.\n\n2. **Effective Python: 90 Specific Ways to Write Better Python by Brett Slatkin**  \n   - **Length**: ~300 pages  \n   - **Difficulty**: Intermediate, with concise tips and practical advice.  \n   - **Estimated Time'

In [47]:
chain_time2.get_graph().print_ascii()

               +-------------------------------+                 
               | Parallel<books,projects>Input |                 
               +-------------------------------+                 
                    ***                  ***                     
                ****                        ****                 
              **                                **               
+--------------------+                   +--------------------+  
| ChatPromptTemplate |                   | ChatPromptTemplate |  
+--------------------+                   +--------------------+  
           *                                        *            
           *                                        *            
           *                                        *            
  +-----------------+                     +-----------------+    
  | AzureChatOpenAI |                     | AzureChatOpenAI |    
  +-----------------+                     +-----------------+    
          

どういうフローで処理をつなげていくのか、頭の中でイメージしながら組んでいく必要がある